In [25]:
import pandas as pd
%matplotlib inline

## Queries

### Query 1

In [20]:
!bq query --use_legacy_sql=false 'SELECT count(*) as total_trips FROM `bigquery-public-data.san_francisco.bikeshare_trips`'

Waiting on bqjob_rd819057e74c6ee1_00000168ad121625_1 ... (0s) Current status: DONE   
+-------------+
| total_trips |
+-------------+
|      983648 |
+-------------+


### Query 2

In [21]:
!bq query --use_legacy_sql=false 'SELECT min(start_date) as Earliest_Start_Time, max(end_date) as Latest_End_Time \
                                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`'

Waiting on bqjob_r7fa645ef6c6f9aab_00000168ad12e5d7_1 ... (0s) Current status: DONE   
+---------------------+---------------------+
| Earliest_Start_Time |   Latest_End_Time   |
+---------------------+---------------------+
| 2013-08-29 09:08:00 | 2016-08-31 23:48:00 |
+---------------------+---------------------+


### Query 3

In [22]:
!bq query --use_legacy_sql=false 'SELECT count(distinct bike_number) as Total_Bikes \
                                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`'

Waiting on bqjob_r563088161f92fc32_00000168ad134341_1 ... (0s) Current status: DONE   
+-------------+
| Total_Bikes |
+-------------+
|         700 |
+-------------+


### Query 4

In [34]:
!bq query --quiet --use_legacy_sql=false 'SELECT count(trip_id) as Morning_Trips \
                                  FROM `bigquery-public-data.san_francisco.bikeshare_trips` \
                                  WHERE TIME(start_date) < "12:00:00"'

+---------------+
| Morning_Trips |
+---------------+
|        412339 |
+---------------+


**Number of morning trips = 412,339**

**Number of afternoon trips = 983648 - 412,339 = 571,309**

## Questions

#### Question 1: How many bike trips were taken by subscribers versus non subscribers?

#### Question 2: How many rides were taken by non-subscribers where the start and end stations are different?

#### Question 3: How many rides were shorter than or equal to 30 mintues? 

#### Question 4: Which are top 5 popular stations where bike rides start in the morning ?

#### Question 5: Which are top 5 popular stations where bike rides start in the afternoon ?

## Answers

#### 1: 

In [40]:
!bq query --quiet --use_legacy_sql=false 'SELECT count(subscriber_type) as Customers \
                                          FROM `bigquery-public-data.san_francisco.bikeshare_trips` \
                                          WHERE (subscriber_type LIKE "Customer")'

+-----------+
| Customers |
+-----------+
|    136809 |
+-----------+


In [41]:
!bq query --quiet --use_legacy_sql=false 'SELECT count(subscriber_type) as Subscribers \
                                          FROM `bigquery-public-data.san_francisco.bikeshare_trips` \
                                          WHERE (subscriber_type LIKE "Subscriber")'

+-------------+
| Subscribers |
+-------------+
|      846839 |
+-------------+


** There are 846,839 annual or monthly bike share subscribers versus 136,809 single or 3 day members **

#### 2: 

In [44]:
!bq query --quiet --use_legacy_sql=false 'SELECT  COUNT(distinct trip_id) as One_Way_Subscriber_Trips\
                                          FROM `bigquery-public-data.san_francisco.bikeshare_trips` \
                                          WHERE start_station_id != end_station_id and subscriber_type = "Subscriber"'

+--------------------------+
| One_Way_Subscriber_Trips |
+--------------------------+
|                   836945 |
+--------------------------+


** A total of 836,945 trips were taken by subscribers that were one way trips **

#### 3: 

In [47]:
!bq query --quiet --use_legacy_sql=false 'SELECT count(distinct trip_id) as Short_Trips \
                                          FROM `bigquery-public-data.san_francisco.bikeshare_trips` \
                                          WHERE (duration_sec) <= 1800'

+-------------+
| Short_Trips |
+-------------+
|      936311 |
+-------------+


#### 4:

In [52]:
!bq query --quiet --use_legacy_sql=false 'SELECT  COUNT(*) as Trip_count, start_station_name\
                                          FROM `bigquery-public-data.san_francisco.bikeshare_trips` \
                                          WHERE TIME(start_date) < "12:00:00" \
                                          GROUP BY start_station_name \
                                          ORDER BY COUNT(start_station_name) DESC LIMIT 5'

+------------+-----------------------------------------------+
| Trip_count |              start_station_name               |
+------------+-----------------------------------------------+
|      47972 | San Francisco Caltrain (Townsend at 4th)      |
|      36450 | San Francisco Caltrain 2 (330 Townsend)       |
|      26413 | Harry Bridges Plaza (Ferry Building)          |
|      23066 | Temporary Transbay Terminal (Howard at Beale) |
|      18667 | Steuart at Market                             |
+------------+-----------------------------------------------+


**The top 5 stations with trips starting in the morning are shown above.**

In [54]:
!bq query --quiet --use_legacy_sql=false 'SELECT  COUNT(*) as Trip_count, start_station_name\
                                          FROM `bigquery-public-data.san_francisco.bikeshare_trips` \
                                          WHERE TIME(start_date) >= "12:00:00" \
                                          GROUP BY start_station_name \
                                          ORDER BY COUNT(start_station_name) DESC LIMIT 5'

+------------+------------------------------------------+
| Trip_count |            start_station_name            |
+------------+------------------------------------------+
|      29566 | Embarcadero at Sansome                   |
|      24798 | 2nd at Townsend                          |
|      24762 | Townsend at 7th                          |
|      24711 | San Francisco Caltrain (Townsend at 4th) |
|      22941 | Market at Sansome                        |
+------------+------------------------------------------+


**The top 5 stations with trips starting in the afternoon are shown above.**